<a href="https://colab.research.google.com/github/zzaeyun/ESAA23_1/blob/main/%EC%99%84%EB%B2%BD%EA%B0%80%EC%9D%B4%EB%93%9C_607to624.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**08 파이썬 추천 시스템 패키지-Surprise**

###**Surprise 패키지 소개**
- 추천 시스템은 상업적으로 가치가 크기 때문에 별도의 패키지로 제공되면 활용도가 높음
- 이전 코드는 최적화나 수행 속도 측면에서 보완이 필요

In [1]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095456 sha256=2830aea88d6053acd6589ecf3d5a00a8dabdff953076ebeb6836967e1a101ef4
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


**Surprise 패키지**
- 다양한 추천 알고리즘, 예를 들어 사용자 또는 아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++, NMF 기반의 잠재 요인 협업 필터링을 쉽게 적용해 추천 시스템 구축 가능
- fit( ), predict( ) API로 추천 데이터 학습과 예측 
- train_test_split( )으로 추천 학습 데이터와 예측 데이터 세트 분리
- cross_validate().GridSearchCV 클래스를 통해 추천 시스템을 위한 모델 셀렉션, 평가, 하이퍼 파라미터 튜닝 등의 기능 제공

###**Surprise를 이용한 추천 시스템 구축**
추천 데이터를 학습용과 테스트용 데이터 세트로 분리한 뒤 SVD 행렬 분해를 통해 잠재 요인 협업 필터링 수행

In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

- Surprise에서 데이터 로딩은 Dataset 클래스를 이용해서만 가능
- Surprise는 Movie Lens 데이터 세트의 사용자-영화 평점 데이터 포맷과 같이 userId(사용자 ID), movieId(영화 ID), rating(평점)과 같은 주요 데이터가 로우 레벨 형태로 돼있는 포맷의 데이터만 처리

In [3]:
#무비렌즈(MovieLens) 사이트에서 제공하는 데이터 세트인 'ml-100k' (10만 개 평점 데이터)
data=Dataset.load_builtin('ml-100k')
trainset, testset=train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


- Surprise가 내려받은 ml-100k, ml-1m은 과거 버전의 데이터 세트
  - 분리 문자가 탭(\t)
- 무비렌즈 사이트에서 내려받은 데이터 파일과 동일하게 로우 레벨의 사용자-아이템 평점 데이터를 그대로 적용해야 함
- Surprise는 자체적으로 로우 레벨의 데이터를 칼럼 레벨의 데이터로 변경하므로 원본인 로우 레벨의 사용자-아이템 평점 데이터를 데이터 세트로 적용

In [4]:
algo=SVD()
algo.fit(trainset)

Surprise에서 추천을 예측하는 메서드는 test()와 predict()
- test(): 사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측하는 메서드, 입력된 데이터 세트에 대해 추천 데이터 세트를 만들어줌
- predict(): 개별 사용자와 영화에 대한 추천 평점을 반환

In [5]:
predictions=algo.test(testset)
print('prediction type: ', type(predictions), len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type:  <class 'list'> 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.786439343718333, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.679179921881626, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.9427123742130163, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.602173872973025, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1298014674277126, details={'was_impossible': False})]

prediction객체
- 개별 사용자 아이디(uid)
- 영화(또는 아이템) 아이디(iid)
- 실제 평점(r_ui)
- 추천 예측 평점(est)
- 데이터를 **튜플** 형태로 가짐
- was_impossible이 True이면 예측값을 생성할 수 없는 데이터라는 의미

In [6]:
[(pred.uid, pred.iid, pred.est ) for pred in predictions[:3]]

[('120', '282', 3.786439343718333),
 ('882', '291', 3.679179921881626),
 ('535', '507', 3.9427123742130163)]

인자로 개별 사용자 아이디, 아이템 아이디를 입력하면 추천 예측 평점을 포함한 정보 반환
- r_ui는 선택 사항
- 사용자 아이디, 아이템 아이디는 문자열로 입력

In [8]:
uid=str(196)
iid=str(302)
pred=algo.predict(uid,iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.04   {'was_impossible': False}


**accuracy** 모듈
- RMSE, MSE 등의 방법으로 추천 시스템의 성능 평가 정보 제공




In [9]:
accuracy.rmse(predictions)

RMSE: 0.9498


0.9498301865128412

###**Surprise 주요 모듈 소개**
**Dataset**
- Surprise는 사용자 아이디, 아이템 아이디, 평점 데이터가 로우 레벨로 된 데이터 세트만 적용 가능
- 네 번째 칼럼부터는 아예 로딩을 수행하지 않음

<br>

API명과 내용
- **Dataset.load_builtin(name='ml-100k')**
  - 무비렌즈 아카이브 FTP 서버에서 데이터를 내려 받음
- **Dataset.load_from_file(file_path, reader)**
  - OS 파일에서 데이터를 로딩할 때 사용
  - 콤마, 탭 등으로 칼럼이 분리된 포맷의 OS 파일에서 데이터를 로딩
- **Dataset.load_from_df(df, reader)**
  - 판다스의 DataFrame에서 데이터를 로딩
  - 파라미터로 DataFrame을 입력받으며 DataFrame 역시 반드시 3개의 칼럼인 사용자 아이디, 아이템 아이디, 평점 순으로 칼럼 순서가 정해져 있어야 함.

**OS 파일 데이터를 Surprise 데이터 세트로 로딩**
- **주의** 로딩되는 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있어서는 안됨

In [11]:
import pandas as pd

ratings=pd.read_csv('/ratings.csv')
ratings.to_csv('/ratings_noh.csv', index=False, header=False)

헤더가 삭제된 파일을 DataSet 모듈의 load_from_file()을 이용해 Dataset로 로드
- 먼저 Reader 클래스를 이용해 데이터 파일의 파싱 포맷을 정의
- 칼럼 헤더가 없고 4개의 칼럼이 콤마로 분리
- Reader 클래스의 생성자에 각 필드의 칼럼명과 칼럼 분리 문자, 그리고 최소~최대 평점을 입력해 객체 생성

In [12]:
from surprise import Reader

reader=Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5)) # 평점의 단위는 0.5, 최대평점은 5
data=Dataset.load_from_file('/ratings_noh.csv', reader=reader)

**Reader 클래스의 주요 생성 파라미터**
- line_format(string): 칼럼을 순서대로 나열, 입력된 문자열을 공백으로 분리해 칼럼으로 인식
- sep(char): 칼럼을 분리하는 분리자, 디폴트는 '\t'
- rating_scale(tuple, optional): 평점 값의 최소~최대 평점을 설정, 디폴트는 (1,5)

In [13]:
trainset, testset=train_test_split(data, test_size=.25, random_state=0 )
algo=SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

**판다스 DataFrame에서 Surprise 데이터 세트로 로딩**
- 사용자 아이디, 아이템 아이디, 평점 칼럼 순서를 지켜야 함

In [16]:
import pandas as pd
from surprise import Reader, Dataset

ratings=pd.read_csv('/ratings.csv')
reader=Reader(rating_scale=(0.5, 5.0))

data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset=train_test_split(data, test_size=.25, random_state=0)

algo=SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

###**Surprise 추천 알고리즘 클래스**
- SVD: 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘
- KNNBasic: 최근접 이웃 협업 필터링을 위한 KNN 알고리즘
- BaselineOnly: 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘

<br>

-Suprise SVD의 비용 함수는 사용자 베이스라인 편향성을 감안한 평점 예측에 Regularization을 적용

<br>

**SVD 클래스의 입력 파라미터**
- n_factors: 잠재 요인 K의 개수, 디폴트는 100, 커질수록 정확도가 높아질 수 있으나 과적합 문제가 발생할 수 있음
- n_epochs: SGD 수행 시 반복 횟수, 디폴트는 20
- biased(bool): 베이스라인 사용자 편향 적용 여부, 디폴트는 True


###**베이스라인 평점**
**베이스라인 평점(Baseline Rating)**
- 개인의 성향을 반영해 아이템 평가에 편향성(bias) 요소를 반영하여 평점을 부과하는 것
- 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수 
  - 전체 평균 평점= 모든 사용자의 평점에 대한 평점을 평균한 값
  - 사용자 편향 점수= 사용자별 아이템 평점 평균 값 - 전체 평균 평점
  - 아이템 편향 점수= 아이템별 평점 평균 값 - 전체 평균 평점 

###**교차 검증과 하이퍼 파라미터 튜닝**
- cross_validate(): 교차 검증
- GridSearchCV: 하이퍼 파라미터 튜닝

<br>

**cross_validate()**
- suprise.model_selection 모듈 내에 존재
- 폴드된 데이터 세트의 개수와 성능 측정 방법을 명시해 교차 검증 수행
- 인자로 알고리즘 객체, 데이터, 성능 평가 방법(measures), 폴드 데이터 세트 개수(cv) 입력

In [17]:
from surprise.model_selection import cross_validate

ratings=pd.read_csv('/ratings.csv')
reader=Reader(rating_scale=(0.5, 5.0))
data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo=SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8727  0.8712  0.8719  0.8774  0.8716  0.8730  0.0023  
MAE (testset)     0.6705  0.6696  0.6691  0.6736  0.6720  0.6710  0.0016  
Fit time          1.85    2.04    2.10    1.72    2.08    1.96    0.15    
Test time         0.14    0.16    0.26    0.30    0.20    0.21    0.06    


{'test_rmse': array([0.87271637, 0.87123971, 0.87186785, 0.87744044, 0.87164393]),
 'test_mae': array([0.67050825, 0.66964769, 0.66910877, 0.67362428, 0.67203197]),
 'fit_time': (1.851870059967041,
  2.036792278289795,
  2.098409652709961,
  1.7246899604797363,
  2.0817227363586426),
 'test_time': (0.13677191734313965,
  0.1576671600341797,
  0.25524020195007324,
  0.30176877975463867,
  0.20180034637451172)}

cross_validate()는 위의 출력 결과와 같이 폴드별 성능 평가 수치와 전체 폴드의 평균 수치를 함께 보여줌

<br>

**GridSearchCV**
- 교차 검증을 통한 하이퍼 파라미터 최적화 수행
- SVD의 경우 주로 점진적 하강 방식의 반복 횟수를 지정하는 n_epochs와 SVD의 잠재 요인 K의 크기를 지정하는 n_factors를 튜닝

In [18]:
from surprise.model_selection import GridSearchCV

param_grid={'n_epochs':[20, 40, 60], 'n_factors':[50, 100, 200]}

gs=GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8778671578290472
{'n_epochs': 20, 'n_factors': 50}


###**Surprise를 이용한 개인화 영화 추천 시스템 구축**
- Surprise를 이용해 잠재 요인 협업 필터링 기반의 개인화된 영화 추천 구현
- 학습된 추천 알고리즘을 기반으로 특정 사용자가 아직 평점을 매기지 않은 (관람하지 않은) 영화 중에서 개인 취향에 가장 적절한 영화를 추천
- Surprise에서는 train_test_split()을 이용해 내부에서 사용하는 TrainSet 클래스 객체로 변환해야만 fit을 통해 학습할 수 있음

In [21]:
#trainset 객체로 변환하지 않아서 오류
'''
data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo=SVD(n_factors=50, random_state=0)
algo.fit(data)
'''

AttributeError: ignored

In [26]:
from surprise.dataset import DatasetAutoFolds

reader=Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
data_folds=DatasetAutoFolds(ratings_file='/ratings_noh.csv', reader=reader)

trainset=data_folds.build_full_trainset()

In [27]:
algo=SVD(n_epochs=10, n_factors=50, random_state=0)
algo.fit(trainset)

In [29]:
movies=pd.read_csv('/movies.csv')
movieIds=ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count()==0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [30]:
uid=str(9)
iid=str(42)

pred=algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.18   {'was_impossible': False}


- 사용자가 평점을 매기지 않은 영화의 추천 예측 평점

<br>

- 사용자가 평점을 매기지 않은 전체 영화를 추출한 뒤에 예측 평점순으로 영화를 추천
- get_unseen_movies()는 사용하지 않고 get_unseen_surprise()함수를 만들고 이를 이용해 평점을 매기지 않은 영화 정보 반환

In [34]:
def get_unseen_surprise(ratings, movies, userId):
  seen_movies=ratings[ratings['userId']==userId]['movieId'].tolist()

  total_movies=movies['movieId'].tolist()

  unseen_movies=[movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수: ', len(seen_movies), '추천 대상 영화 수: ', len(unseen_movies), '전체 영화 수: ',len(total_movies))

  return unseen_movies

unseen_movies=get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수:  46 추천 대상 영화 수:  9696 전체 영화 수:  9742


- recomm_movie_by_surprise()
  - 인자로 학습이 완료된 추천 알고리즘 객체, 추천 대상 사용자 아이디, 추천 대상 영화의 리스트 객체, 그리고 추천 상위 N개 개수

In [37]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
  predictions=[algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  def sortkey_est(pred):
    return pred.est

  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions=predictions[:top_n]

  top_movie_ids=[int(pred.iid) for pred in top_predictions]
  top_movie_rating=[pred.est for pred in top_predictions]
  top_movie_titles=movies[movies.movieId.isin(top_movie_ids)]['title']

  top_movie_preds=[(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_rating, top_movie_titles)]

  return top_movie_preds

In [38]:
unseen_movies=get_unseen_surprise(ratings, movies, 9)
top_movie_preds=recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

평점 매긴 영화 수:  46 추천 대상 영화 수:  9696 전체 영화 수:  9742


In [39]:
print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
  print(top_movie[1], ':', top_movie[2])

##### Top-10 추천 영화 리스트 #####
4.13314590038047 : Usual Suspects, The (1995)
4.1169758732420245 : Star Wars: Episode IV - A New Hope (1977)
4.08608979877929 : Pulp Fiction (1994)
4.080177009680638 : Shawshank Redemption, The (1994)
4.067961488164734 : Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)
4.045129163079612 : Godfather, The (1972)
4.044279846912132 : Rear Window (1954)
4.014277246697982 : Star Wars: Episode V - The Empire Strikes Back (1980)
4.011489026290009 : Goodfellas (1990)
4.008193516308413 : Fight Club (1999)
